In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
from keras.datasets import mnist, cifar10
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from IPython.display import clear_output

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train= np.concatenate([x_train, x_test])
x_train.shape

(70000, 28, 28)

In [3]:
def PSNR(I,Iref):
    mse=np.mean((Iref-I)**2)
    if mse==0:#MSE nul veut dire qu'il n'y a pas de bruit dans le signal donc psnr sans importance
        return 10000
    max_pixel=255
    psnr=np.exp((max_pixel/np.sqrt(mse)))/10**9
    return psnr

In [4]:
#création d'une matrice de distance des images en utilisant le PSNR comme métrique
def mat_sim(data):
    n=len(data)
    mat_psnr=np.zeros((n,n))
    for i in range (n):
        if (i%100)==0 :
            clear_output(wait=True)
            print((i/len(xcut))*100)
        for j in range (i+1,n):
            mat_psnr[i,j]=PSNR(x_train[j],x_train[i])
            mat_psnr[j,i]=PSNR(x_train[j],x_train[i])
    return mat_psnr

In [7]:
xcut=x_train[:100,:]
xcut.shape

(100, 28, 28)

In [8]:
mat_dist=-(mat_sim(xcut))

0.0


In [9]:
mat_dist.astype(int).shape

(100, 100)

In [10]:
list_taille_clusters=np.arange(1,160,4)

In [34]:
#utilisaion de l'agglomerative clustering pour compute la matrice de distance
list_clusters_ac=[]
j=0
for i in list_taille_clusters:
    clear_output(wait=True)
    print((j/len(list_taille_clusters))*100)
    ac = AgglomerativeClustering(n_clusters=i, affinity='precomputed',linkage='average')
    clusters = ac.fit_predict((mat_dist.astype(int)))
    list_clusters_ac+= [clusters]
    j+=1
    

97.5


In [35]:
for i in range (len(list_clusters_ac)):
    path=".\Clusters\PSNR"+list_taille_clusters[i].astype(str) +".npy "
    np.save(path, list_clusters_ac[i])